## Data Collecting and Cleaning
The purpose of this notebook is:
- to collect beer reviews data from kaggle
- do an inital clean of the data 
- save the newly cleaned data in a database in SQL on my AWS to access in future notebooks

***
### Check that AWS is set up properly:
1. open aws terminal:  
    - ` ssh ubuntu@myaws`  
    
    
2. type following comands:  
   -  `sudo su - postgres`  
   -  `nano /etc/postgresql/10/main/pg_hba.conf`  
   
   
3. search for "IPv4 local connections"
    - control + W to search
  
  
4. second line with your IP should match AWS instance 
    - go to AWS instance
    - go to my inbound rules 
    - PostgreSQL -> MY IP -> IP that need to be in nano file 
    
    
5. change file if needed
    - control + X to exit
    - click Y to save 
    - press enter 
    
6. exit the user from being `postgre`  
    - just type "exit"
    
    
7. We have updated the config files, but now we need to get postgreSQL to restart for the changes to take effect. In the terminal type
     - `sudo service postgresql restart`

*** 
### Getting dataset from Kaggle onto AWS:
1. On local computer:
    - have kaggle API installed 
    - type this to download zip onto local home:  
        `~/.local/bin/kaggle datasets download -d rdoume/beerreviews`

    
2. Copy the zip to remote AWS machine 
    - make sure you are where the file is
    - type this to make the copy over:  
        `scp beerreviews.zip ubuntu@myaws:/home/ubuntu`

    
    
3. in AWS unzip the file and move it:  
    - `sudo apt-get install unzip`  
    - `mkdir beerdata`  
    - `unzip beerreviews.zip -d beerdata`
    - `rm beerreviews.zip` 

***
### Create datebase for beer reviews

In `psql` on AWS terminal


``` sql

ubuntu= CREATE DATABASE beerdata;
ubuntu= \connect beerdata

```

***

### Importing Libaries:
Need to import the libaries to be used in this notebook

In [1]:
## Get pandas and postgres to work together
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql

***
### Connecting to AWS
We set up the database on AWS above. To connect, need the public IP address of AWS instance.

In [2]:
# Postgres info to connect

connection_args = {
    'host': '52.52.34.207',  # You will need to put the PUBLIC IP address of your AWS instance here (might have changed might have no)
    'user': 'ubuntu',        # your username on AWS is 'ubuntu'
    'dbname': 'beerdata',    # DB that we are connecting to
    'port': 5432             # port we opened on AWS
}

connection = pg.connect(**connection_args)

In [3]:
cursor = connection.cursor()

In [7]:
query_build_table = """
        CREATE TABLE IF NOT EXISTS beer_reviews (
        brewery_id double precision DEFAULT NULL,
        brewery_name text DEFAULT NULL,
        review_time double precision DEFAULT NULL,
        review_overall double precision DEFAULT NULL,
        review_aroma double precision DEFAULT NULL,
        review_appearance double precision DEFAULT NULL,
        review_profilename text DEFAULT NULL, 
        beer_style text DEFAULT NULL,
        review_palate double precision DEFAULT NULL,
        review_taste double precision DEFAULT NULL,
        beer_name text DEFAULT NULL,
        beer_abv double precision DEFAULT NULL,
        beer_beerid double precision DEFAULT NULL
    );
"""

In [8]:
cursor.execute(query_build_table)

In [9]:
cursor.execute('commit;')

In [10]:
cursor.execute('BEGIN;')

In [11]:
query_copy = "COPY beer_reviews FROM '/home/ubuntu/beerdata/beer_reviews.csv' DELIMITER ',' CSV HEADER;"

In [12]:
cursor.execute(query_copy)

In [13]:
cursor.execute('commit;')

In [14]:
query = "SELECT * FROM beer_reviews;"

beer_reviews_df = pd_sql.read_sql(query, connection)

***
### Clean data using pandas:
- read in csv from AWS 
- clean data 
- save as new cleaned data file 

In [15]:
beer_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null float64
brewery_name          1586614 non-null object
review_time           1586614 non-null float64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
bee_abv               1518829 non-null float64
beer_beerid           1586614 non-null float64
dtypes: float64(9), object(4)
memory usage: 157.4+ MB


In [16]:
beer_reviews_df.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,bee_abv,beer_beerid
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06,1.586614e+06
mean,3.130099e+03,1.224089e+09,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00,2.171279e+04
std,5.578104e+03,7.654427e+07,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00,2.181834e+04
min,1.000000e+00,8.406720e+08,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,3.000000e+00
25%,1.430000e+02,1.173224e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.717000e+03
50%,4.290000e+02,1.239203e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.390600e+04
75%,2.372000e+03,1.288568e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.944100e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731700e+04


In [18]:
# beer_abv has many null values and is not needed for a feature 
# remove column before dropna to prevent from dropping excess rows 
beer_reviews_df = beer_reviews_df.drop(columns = ['beer_abv'])

In [19]:
# check that some types aren't different by factors other than their name
# could I combine similar ones? i.e. American Amber having 2 types -> turn into one 
beer_types = beer_reviews_df.beer_style.unique()
print(sorted(beer_types))

['Altbier', 'American Adjunct Lager', 'American Amber / Red Ale', 'American Amber / Red Lager', 'American Barleywine', 'American Black Ale', 'American Blonde Ale', 'American Brown Ale', 'American Dark Wheat Ale', 'American Double / Imperial IPA', 'American Double / Imperial Pilsner', 'American Double / Imperial Stout', 'American IPA', 'American Malt Liquor', 'American Pale Ale (APA)', 'American Pale Lager', 'American Pale Wheat Ale', 'American Porter', 'American Stout', 'American Strong Ale', 'American Wild Ale', 'Baltic Porter', 'Belgian Dark Ale', 'Belgian IPA', 'Belgian Pale Ale', 'Belgian Strong Dark Ale', 'Belgian Strong Pale Ale', 'Berliner Weissbier', 'Bière de Champagne / Bière Brut', 'Bière de Garde', 'Black & Tan', 'Bock', 'Braggot', 'California Common / Steam Beer', 'Chile Beer', 'Cream Ale', 'Czech Pilsener', 'Doppelbock', 'Dortmunder / Export Lager', 'Dubbel', 'Dunkelweizen', 'Eisbock', 'English Barleywine', 'English Bitter', 'English Brown Ale', 'English Dark Mild Ale', '

Since there are so many beer types that are very similar to each other, here we are going to group types together to simplify the classification. Doing some reseach this is the grouping I have come up with:

Note that not all of the origional types fit into one of thse groups so will keep their origional name. These are adressed in the following code block. 

In [20]:
Belgian_Style = ['Dubbel', 'Quadrupel (Quad)','Tripel','Saison / Farmhouse Ale']
for i in Belgian_Style:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Belgian Style'

Bock = ['Bock', 'Doppelbock','Eisbock','Maibock / Helles Bock','Weizenbock']
for i in Bock:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Bock'

Brown_Ale = ['American Brown Ale', 'English Brown Ale', 'English Dark Mild Ale','Altbier']
for i in Brown_Ale:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Brown Ale'

Dark_Ale = ['Belgian Dark Ale','Belgian Strong Dark Ale']
for i in Dark_Ale:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Dark Ale'

Dark_Lager = ['Euro Dark Lager', 'Munich Dunkel Lager','American Amber / Red Lager', 'Schwarzbier',
              'Märzen / Oktoberfest','Vienna Lager', 'Euro Dark Lager','Munich Dunkel Lager']
for i in Dark_Lager:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Dark Lager'

Hybrid = ['California Common / Steam Beer', 'Flanders Red Ale', 'Irish Red Ale', 'Bière de Garde','Cream Ale'] 
for i in Hybrid:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Hybrid'

IPA = ['American IPA', 'Belgian IPA','American Double / Imperial IPA','English India Pale Ale (IPA)']    
for i in IPA:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'IPA'

Pale_Ale = ['Belgian Pale Ale','American Pale Wheat Ale','American Pale Ale (APA)','English Pale Ale',
            'Belgian Strong Pale Ale','American Blonde Ale','Extra Special / Strong Bitter (ESB)','English Bitter']
for i in Pale_Ale:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Pale Ale'

Pilsner = ['Czech Pilsener', 'American Double / Imperial Pilsner','German Pilsener']   
for i in Pilsner:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Pilsner'

Pale_Lager =['American Adjunct Lager','Euro Pale Lager','American Pale Lager','Munich Helles Lager',
             'Dortmunder / Export Lager','Light Lager','English Pale Mild Ale']
for i in Pale_Lager:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Pale Lager'

Porter = ['American Porter', 'Baltic Porter']
for i in Porter:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Porter'

Rye_Beer = ['Roggenbier','Rye Beer']
for i in Rye_Beer:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Rye Beer'

Smoked_Beer = ['Rauchbier', 'Smoked Beer']
for i in Smoked_Beer:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Smoked Beer'
    
Scottish_Style_Ale = ['Scotch Ale / Wee Heavy', 'Scottish Ale','Scottish Gruit / Ancient Herbed Ale']
for i in Scottish_Style_Ale:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Scottish Style Ale'

Stout = ['American Stout','American Double / Imperial Stout','English Stout','Foreign / Export Stout',
         'Irish Dry Stout','Milk / Sweet Stout','Oatmeal Stout', 'Russian Imperial Stout']
for i in Stout:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Stout'

Strong_Ale = ['American Strong Ale', 'English Strong Ale','American Barleywine','English Barleywine','Old Ale',
              'American Amber / Red Ale']
for i in Strong_Ale:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Strong Ale'

Wheat_Beer = ['Hefeweizen', 'Kristalweizen', 'Witbier','Dunkelweizen', 'American Dark Wheat Ale']
for i in Wheat_Beer:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Wheat Beer'
    
Wild_Sour = ['Berliner Weissbier', 'Gose', 'Lambic - Fruit', 'Lambic - Unblended', 'Flanders Oud Bruin', 
             'American Wild Ale','Gueuze', 'Faro']   
for i in Wild_Sour:
    beer_reviews_df.loc[beer_reviews_df['beer_style'].str.contains(i), 'beer_type'] = 'Wild/Sour'   

/Users/sammy/miniconda3/envs/metis/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [21]:
beer_reviews_df['beer_type'] = beer_reviews_df['beer_type'].fillna(beer_reviews_df['beer_style'])

In [22]:
beer_reviews_df = beer_reviews_df.dropna()

In [23]:
beer_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1586266 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586266 non-null float64
brewery_name          1586266 non-null object
review_time           1586266 non-null float64
review_overall        1586266 non-null float64
review_aroma          1586266 non-null float64
review_appearance     1586266 non-null float64
review_profilename    1586266 non-null object
beer_style            1586266 non-null object
review_palate         1586266 non-null float64
review_taste          1586266 non-null float64
beer_name             1586266 non-null object
beer_beerid           1586266 non-null float64
beer_type             1586266 non-null object
dtypes: float64(8), object(5)
memory usage: 169.4+ MB


In [25]:
beer_reviews_df.to_csv('cleaned_data.csv', index=False)
# turned into csv to be copied into postgres table 
# next time: find a way to go straight from DF to postgres table 

***
### Create table in data base for beer reviews (the cleaned data)
- Take the newly cleaned data and place it on AWS:
    - On local terminal:   
        `scp cleaned_data.csv ubuntu@myaws:/home/ubuntu`  
        `rm cleaned_data.csv`

    - on AWS terminal:  
        `mv cleaned_data.csv beerdata` 
        
- Now that the csv file is on AWS copy the data to a datatable in the database 


In [31]:
cursor.execute('BEGIN;')

query_build_table = """
        CREATE TABLE IF NOT EXISTS beer (
        brewery_id double precision DEFAULT NULL,
        brewery_name text DEFAULT NULL,
        review_time double precision DEFAULT NULL,
        review_overall double precision DEFAULT NULL,
        review_aroma double precision DEFAULT NULL,
        review_appearance double precision DEFAULT NULL,
        review_profilename text DEFAULT NULL, 
        beer_style text DEFAULT NULL,
        review_palate double precision DEFAULT NULL,
        review_taste double precision DEFAULT NULL,
        beer_name text DEFAULT NULL,
        beer_beerid double precision DEFAULT NULL, 
        beer_type text DEFAULT NULL
    );
"""

In [32]:
cursor.execute(query_build_table)

In [33]:
cursor.execute('commit;')

In [34]:
cursor.execute('BEGIN;')

In [35]:
query_copy = "COPY beer FROM '/home/ubuntu/beerdata/cleaned_data.csv' DELIMITER ',' CSV HEADER;"

In [36]:
cursor.execute(query_copy)

In [37]:
cursor.execute('commit;')

***
### Getting averages 
- using the cursor methond, can play with the data in SQL
- get users average rating for each beer types
- get users average rating for each brewery 
- NOTE: this is done here in SQL, but also done slightly difference using pandas in the next notebook 

#### create a table for the users average reviews for each beer type:

In [38]:
cursor.execute('BEGIN;')

query_build_table = """
        CREATE TABLE IF NOT EXISTS user_types (
        user_name text DEFAULT NULL, 
        beer_style text DEFAULT NULL,
        review_count double precision DEFAULT NULL,
        avg_review double precision DEFAULT NULL,
        avg_apperance double precision DEFAULT NULL,
        avg_palate double precision DEFAULT NULL,
        avg_taste double precision DEFAULT NULL,
        avg_aroma double precision DEFAULT NULL
    );
"""

In [39]:
cursor.execute(query_build_table)

In [40]:
cursor.execute('commit;')

In [41]:
cursor.execute('BEGIN;')

query_types = """
    INSERT INTO user_types
    SELECT review_profilename as user_name, beer_style, 
            COUNT(review_overall) as review_count,
            AVG(review_overall) as avg_review,
            AVG(review_appearance) as avg_apperance,
            AVG(review_palate) as avg_palate,
            AVG(review_taste) as avg_taste,
            AVG(review_aroma) as avg_aroma
    FROM beer
    GROUP BY review_profilename, beer_style
"""

In [42]:
cursor.execute(query_types)

In [43]:
cursor.execute('commit;')

#### create a table for the users average reviews for each brewery:

In [44]:
query_build_table = """
        CREATE TABLE IF NOT EXISTS user_breweries (
        user_name text DEFAULT NULL, 
        brewery_name text DEFAULT NULL,
        review_count double precision DEFAULT NULL,
        avg_review double precision DEFAULT NULL,
        avg_apperance double precision DEFAULT NULL,
        avg_palate double precision DEFAULT NULL,
        avg_taste double precision DEFAULT NULL,
        avg_aroma double precision DEFAULT NULL
    );
"""

In [45]:
cursor.execute(query_build_table)

In [46]:
cursor.execute('commit;')

In [47]:
# Since we have committed already we should start a transaction explicitly.
cursor.execute('BEGIN;')

query_brewery = """
    INSERT INTO user_breweries 
    SELECT review_profilename as user_name, brewery_name, 
            COUNT(review_overall) as review_count,
            AVG(review_overall) as avg_review,
            AVG(review_appearance) as avg_apperance,
            AVG(review_palate) as avg_palate,
            AVG(review_taste) as avg_taste,
            AVG(review_aroma) as avg_aroma
    FROM beer
    GROUP BY review_profilename, brewery_name
"""

In [48]:
cursor.execute(query_brewery)

In [49]:
cursor.execute('commit;')

*** 
User features to make:
   - total number of reviews the user has done: user_total_review_count  
   
    *user-type features -*  
   - number of reviews for each type by user: user_type_review_count
   - ratio of reviews of type compared to total reviews: user_review_type_ratio 
   - number of types of beers user has reviewed: user_review_number_of_types  
   - average overall rating for each beer type: user_avg_overall_rating_of_type
   - average apperance rating for each beer type: user_avg_apperance_rating_of_type
   - average aroma rating for each beer type: user_avg_aroma_rating_of_type
   
   *user-beer features -*
   - number of reviews for each type by user: user_beer_review_count
   - ratio of reviews of type compared to total reviews: user_review_beer_ratio 
   - number of types of beers user has reviewed: user_review_number_of_beers  
   - average overall rating for each beer type: user_avg_overall_rating_of_beer 
   
   *user-brewery features -*  
   - number of breweries user has reviewed: user_review_number_of_breweries
   - number of reviews for each brewery by user: user_brewery_review_count
   - ratio of reviews of brewery compared to total: user_review_brewery_ratio
   - average overall rating for each brewery: user_avg_overall_rating
        


Beer features to make:
   - total number of reviews for each beer: beer_total_review_count
   - average overall rating for each beer: beer_avg_rating
    
 
 
Beer type features to make:
   - total number of reviews for each beer type: type_total_review_count
   - average overall rating for each type: type_avg_rating 


Brewery features to make:
   - total number of reviews for each brewery: brewery_total_review_count
   - average overall rating for each brewery: brewery_avg_overall_rating 